In [1]:
from seabreeze.spectrometers import list_devices, Spectrometer
import pylablib as pll
pll.par["devices/dlls/thorlabs_tlcam"] = "path/to/dlls"
from pylablib.devices import Thorlabs
from threading import Lock

import random
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import keyboard
import h5py
import time

file_storage=r'C:\Users\tl457\OneDrive - University Of Cambridge 1\3_Code\lwel-control\test.h5'
cam_serial='16906'

In [2]:
# abstract base class to represent camera
class CameraSpec:
    def __init__(self, camLock, commLock1, specLock, commLock2):
        self._camera = None                # spectrometer
        self._camModel = ''              # model name for graph title
        self._camData = [[], []]     # wavelengths and intensities
        self._controlFunctions = {}       # behaviour upon changing controls
        self._roi_x_limit = (0,1)    # x ROI limits
        self._roi_y_limit = (0,1)     # y ROI limits
        self.roi=(100,1440,4,1080,1,1)
        self.comm1_lock = commLock1        # for communicating with camera
        self.cam_lock = camLock         # for editing spectrometer values
        self._spec = None                 # spectrometer
        self._specModel = ''              # model name for graph title
        self._spectralData = [[], []]     # wavelengths and intensities
        self._controlFunctions = {}       # behaviour upon changing controls
        self._int_time_max = 1    # maximum integration time (ms)
        self._int_time_min = 0         # minimum integration time (ms)
        self.comm2_lock = commLock2         # for communicating with spectrometer
        self.spec_lock = specLock         # for editing spectrometer values
        self.spec_int_time=1
        self.cam_int_time=1

    # refreshes/populates camera properties
    def assign_cam(self):
        return

    # get data for graph
    def get_image(self):
        return self._camData

    # send each command; return successes and failures
    def send_control_values(self, commands):
        return ({}, {})
    
    # getter methods
    def model_cam(self):
        return self._camModel

    def roi_x_limit(self):
        return self._roi_x_limit

    def roi_y_limit(self):
        return self._roi_y_limit
    
    # refreshes/populates spectrometer properties
    def assign_spec(self):
        return

    # get data for graph
    def get_spectrum(self):
        return self._spectralData
    
    # getter methods
    def model_spec(self):
        return self._specModel
    
    def int_time_max(self):
        return self._int_time_max

    def int_time_min(self):
        return self._int_time_min
    
class PhysicalCameraSpec(CameraSpec):
    
    def __init__(self, camLock, commLock1, specLock, commLock2):
        super().__init__(camLock, commLock1, specLock, commLock2)
        self.cam_lock.acquire()
        self.assign_cam()
        self.cam_lock.release()
        self.spec_lock.acquire()
        self.assign_spec()
        self.spec_int_time=0.5
        self.cam_int_time=0.1
        self.spec_lock.release()
        self._controlFunctions = {
            'cam-integration-time-input':
            "self._cam.set_exposure",
            'roi-input':
            "self._cam.set_roi",
            'frame-rate-input':
            "self._cam.set_frame_period",
            'spec-integration-time-input':
            "self._spec.integration_time_micros",
            
        }
        self.taking_images=False
        self.acquisition_ready=False
        self.taking_spectra=False
        self.number_measure=5
        self.time_interval_seconds=1

    def assign_cam(self,serial_id='16906'):
        self.comm1_lock.acquire()
        devices = Thorlabs.TLCamera.list_cameras()
        self._cam = Thorlabs.ThorlabsTLCamera(serial=serial_id)
        self._camModel = self._cam.get_device_info()[0]
        self._cam.set_exposure(self.cam_int_time)
        self._cam.set_roi(100,1440,4,1080,1,1)
        self._roi_x_limit = (self._cam.get_roi_limits()[0][0],self._cam.get_roi_limits()[0][1])
        self._roi_y_limit = (self._cam.get_roi_limits()[1][0],self._cam.get_roi_limits()[1][1])
        print('Camera '+str(self._camModel)+' connected with roi limits '+str(self._roi_x_limit)+' and '+str(self._roi_y_limit))
        print(self._cam.get_roi(),self._cam.get_frame_timings())
        self._cam.close()
        self.comm1_lock.release()
            
    def assign_spec(self):
        self.comm2_lock.acquire()
        devices = list_devices()
        self._spec = Spectrometer(devices[0])
        self._specmodel = self._spec.model
        self._int_time_min = self._spec.integration_time_micros_limits[0]
        self._int_time_max = self._spec.integration_time_micros_limits[1]
        self.comm2_lock.release()
        print('Spectrometer '+str(self._specmodel)+' connected with integration limits '+str(int(self._int_time_min))+' to '+str(int(self._int_time_max)))

    def get_image(self):
        if self._cam is None:
            try:
                self.cam_lock.acquire()
                self.assign_cam()
            except Exception:
                pass
            finally:
                self.cam_lock.release()
        try:
            self.comm1_lock.acquire()
            self._cam.open()
            self._cam.set_exposure(self.cam_int_time)
            image=self._cam.grab(1)
            self._camData = image[0]
        except Exception:
            pass
        finally:
            self._cam.close()
            self.comm1_lock.release()

        return self._camData
    
    def get_spectrum(self):
        if self._spec is None:
            try:
                self.spec_lock.acquire()
                self.assign_spec()
            except Exception:
                pass
            finally:
                self.spec_lock.release()
        try:
            self.comm2_lock.acquire()
            self._spectralData = self._spec.spectrum(correct_nonlinearity=True)
        except Exception:
            pass
        finally:
            self.comm2_lock.release()

        return self._spectralData
    
    def setup_acquisition(self):
        if self._cam is None:
            try:
                self.cam_lock.acquire()
                self.assign_cam()
            except Exception:
                pass
            finally:
                self.cam_lock.release()
        try:
            self.comm1_lock.acquire()
            self._cam.open()
            self._cam.setup_acquisition(nframes=100)
            print(self._cam.get_acquisition_parameters())
            self.acquisition_ready=True
            
        except Exception:
            pass
        
        finally:
            self._cam.close()
            self.comm1_lock.release()

    
    def start_acquisition(self):
        if self.acquisition_ready==False:
            try:
                self.cam_lock.acquire()
                self.setup_acquisition()
            except Exception:
                pass
            finally:
                self.cam_lock.release()
        N=0
        self.comm1_lock.acquire()
        self._cam.open()
        self.taking_images=True
        self.taking_spectra = True
        f=h5py.File(file_storage, "w")
        f.attrs.create("time_interval", self.time_interval_seconds)
        f.attrs.create("cam_integration_time", self.cam_int_time)
        f.attrs.create("spec_integration_time", self.spec_int_time)
        timestamp=datetime.now()
        f.attrs.create("creation_time", timestamp.strftime("%Y-%m-%dT%H:%M:%S.%f"))
        f.close()
        self._cam.start_acquisition()
        while N < self.number_measure and self.taking_images and self.taking_spectra:
            if N!=0: # starting data collection immediately
                time.sleep(self.time_interval_seconds) 
            f = h5py.File(file_storage, 'a')
            grp = f.create_group("dataset_%d" % N)
            self._cam.wait_for_frame()  # wait for the next available frame
            frame = self._cam.read_oldest_image()  # get the oldest image which hasn't been read yet
            arr = self.get_spectrum()
            grp.create_dataset("image_%d" % N, data=frame)
            grp.create_dataset("spec_%d" % N, data=np.vstack((arr[0],arr[1])))
            timestamp=datetime.now()
            grp.attrs['timestamp'] = timestamp.strftime("%Y-%m-%dT%H:%M:%S.%f")
            f.close()
            N += 1
            print("Spectrum and image %d of %d recorded" % (N,self.number_measure))
        print("Done!\n")
        self._cam.stop_acquisition()
        self._cam.close()
        self.taking_images=False
        self.taking_spectra = False
        self.comm1_lock.release()

    def send_control_values(self, commands):
        failed = {}
        succeeded = {}
        
        for ctrl_id in commands:
            try:
                self.comm_lock.acquire()
                eval(self._controlFunctions[ctrl_id])(commands[ctrl_id])
                succeeded[ctrl_id] = str(commands[ctrl_id])
            except Exception as e:
                failed[ctrl_id] = str(e).strip('b')
            finally:
                self.comm_lock.release()
                
        return(failed, succeeded)
            
    def cam_model(self):
        self.cam_lock.acquire()
        self._cam.open()
        self.assign_cam()
        self._cam.close()
        self.cam_lock.release()
        return self._camModel
    
    def roi_x_limit(self):
        self.cam_lock.acquire()
        self._cam.open()
        self.assign_cam()
        self._cam.close()
        self.cam_lock.release() 
        return self._roi_x_limit

    def roi_y_limit(self):
        self.cam_lock.acquire()
        self._cam.open()
        self.assign_cam()
        self._cam.close()
        self.cam_lock.release() 
        return self._roi_y_limit
    
    def spec_model(self):
        self.spec_lock.acquire()
        self.assign_spec()
        self.spec_lock.release()
        return self._specmodel

    def int_time_max(self):
        self.spec_lock.acquire()
        self.assign_spec()
        self.spec_lock.release() 
        return self._int_time_max

    def int_time_min(self):
        self.spec_lock.acquire()
        self.assign_spec()
        self.spec_lock.release()  
        return self._int_time_min
    

In [3]:
#############################
# Device properties
#############################

# lock for modifying information about spectrometer
cam_lock = Lock()
# lock for communicating with spectrometer
comm_lock1 = Lock()
# lock for modifying information about spectrometer
spec_lock = Lock()
# lock for communicating with spectrometer
comm_lock2 = Lock()

dev = PhysicalCameraSpec(cam_lock, comm_lock1, spec_lock, comm_lock2)
dev.assign_cam()
dev.assign_spec()
dev.setup_acquisition()

Camera CS165CU connected with roi limits (80, 1440) and (4, 1080)
(100, 1440, 4, 1080, 1, 1) TAcqTimings(exposure=1.000001, frame_period=1.000089)
Spectrometer FLAMEX connected with integration limits 1000 to 60000000
Camera CS165CU connected with roi limits (80, 1440) and (4, 1080)
(100, 1440, 4, 1080, 1, 1) TAcqTimings(exposure=0.100008, frame_period=0.10009599999999999)
Spectrometer FLAMEX connected with integration limits 1000 to 60000000
{'nframes': 100}


In [4]:
dev.start_acquisition()

Spectrum and image 1 of 5 recorded
Spectrum and image 2 of 5 recorded
Spectrum and image 3 of 5 recorded
Spectrum and image 4 of 5 recorded
Spectrum and image 5 of 5 recorded
Done!

